In [2]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from model2 import Generator
from dataset2 import APOGEEDataset
from utils import resolve_path, get_config
from torch.utils.data import DataLoader
import os


In [4]:
# '../latent/latent_codes_epoch_1.npy'
# Load configuration and set paths
config = get_config()
data_path = resolve_path(config['paths']['hdf5_data'])
latent_path = resolve_path(config['paths']['latent'])
checkpoint_path = resolve_path(config['paths']['checkpoints'])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the latent codes
latent_file = os.path.join(latent_path, 'latent_codes_epoch_99.npy')  
latent_data = np.load(latent_file, allow_pickle=True).item()
latent_codes = latent_data['latent_codes']
indices = latent_data['indices']

# Get the latent code for spectrum index 9
spectrum_index = 9
latent_code = torch.tensor(latent_codes[indices == spectrum_index]).to(device)


In [5]:

# Initialize the generator model
generator = Generator(config['training']['latent_dim'], config['model']['output_dim'], config['model']['generator_layers'], config['model']['activation_function']).to(device)
checkpoint = torch.load(os.path.join(checkpoint_path, 'checkpoint_latest.pth.tar'), map_location=device)  # Change 'latest' to the actual checkpoint file if needed
generator.load_state_dict(checkpoint['state_dict'])
generator.eval()


Generator(
  (model): Sequential(
    (0): Linear(in_features=5, out_features=256, bias=True)
    (1): LeakyReLU(negative_slope=0.2)
    (2): Linear(in_features=256, out_features=512, bias=True)
    (3): LeakyReLU(negative_slope=0.2)
    (4): Linear(in_features=512, out_features=1024, bias=True)
    (5): LeakyReLU(negative_slope=0.2)
    (6): Linear(in_features=1024, out_features=512, bias=True)
    (7): LeakyReLU(negative_slope=0.2)
    (8): Linear(in_features=512, out_features=256, bias=True)
    (9): LeakyReLU(negative_slope=0.2)
    (10): Linear(in_features=256, out_features=8575, bias=True)
  )
)

In [6]:

# Generate the fake spectrum
with torch.no_grad():
    fake_spectrum = generator(latent_code.unsqueeze(0)).squeeze(0).cpu().numpy()


In [8]:

# Load the real spectrum
dataset = APOGEEDataset(data_path, max_files=100)
data_loader = DataLoader(dataset, batch_size=1, shuffle=False, num_workers=0)

for batch in data_loader:
    if batch['index'].item() == spectrum_index:
        real_spectrum = batch['flux'].squeeze(0).cpu().numpy()
        break


In [10]:
for batch in data_loader:
    print(batch['index'].item())

3.0
5.0
2.0
5.0
9.0
9.0
6.0
7.0
9.0
5.0
2.0
6.0
6.0
8.0
8.0
5.0
7.0
2.0
6.0
3.0
5.0
0.0
9.0
5.0
4.0
2.0
9.0
4.0
8.0
2.0
0.0
5.0
4.0
5.0
2.0
3.0
1.0
3.0
7.0
3.0
3.0
8.0
0.0
5.0
1.0
8.0
5.0
5.0
7.0
8.0
2.0
6.0
1.0
8.0
1.0
0.0
2.0
5.0
4.0
0.0
6.0
9.0
6.0
0.0
9.0
5.0
7.0
1.0
8.0
2.0
0.0
1.0
6.0
0.0
0.0
5.0
5.0
3.0
2.0
6.0
3.0
4.0
2.0
8.0
1.0
6.0
0.0
8.0
4.0
7.0
2.0
1.0
9.0
7.0
8.0
1.0
5.0
3.0


KeyboardInterrupt: 

In [11]:

# Plot both spectra
plt.figure(figsize=(10, 5))
plt.plot(real_spectrum, label='Real Spectrum', color='blue', alpha=0.8, linewidth=0.8)
plt.plot(fake_spectrum, label='Fake Spectrum', color='red', alpha=0.8, linewidth=0.8)
plt.title(f'Comparison of Real and Generated Spectra for Spectrum Index {spectrum_index}')
plt.xlabel('Wavelength Index')
plt.ylabel('Flux')
plt.legend()
plt.grid(True)
plt.show()


/opt/conda/lib/python3.11/site-packages/IPython/core/pylabtools.py:152: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  fig.canvas.print_figure(bytes_io, **kw)


ValueError: Image size of 846x179670 pixels is too large. It must be less than 2^16 in each direction.

<Figure size 1000x500 with 1 Axes>